# E65 analysis with relaxed Norm params

Here we will repeat the previous analysis but only progressing past the normalisation when we can see our marker genes of interest.

In [3]:
# Load scripts and source data
suppressPackageStartupMessages(
    source("source/RaceID3_StemID2_class.R")
) 

x <- read.csv("source/E725.matrix.Seb_NewData_E725.3.quantif",sep="\t",header=TRUE, row.name=1)
prdata <- x[grep("ERCC",rownames(x),invert=TRUE),]

In [4]:
# Check that our raw data and filtered out ERCC data match dimensions
dim(x)
dim(prdata)


[1] 49671   192

[1] 49671   192

In [5]:
sc <- SCseq(prdata)
message(dim(sc@fdata)[[1]], " genes and ", dim(sc@fdata)[[2]], " cells.")

49671 genes and 192 cells.


# Obtain a full Biomart

Let's get a biomart of ALL genes in our data, instead of just the filtered ones, and then we will test it against our genes of interest to see if they still persist after filtering.

In [7]:
require(biomaRt)
# Annotate the data before adding SC experiment

mart <- useMart(biomart = "ENSEMBL_MART_ENSEMBL", dataset = "mmusculus_gene_ensembl")

Loading required package: biomaRt


In [ ]:
k <- getBM(
    filters = "ensembl_gene_id",
    attributes = c(
            "ensembl_gene_id",              # Gene stable ID
            "external_gene_name",           # Casual name
#            "external_transcript_name",     # Transcript-specific name
#            "gene_biotype",                 # Gene biotype
#            "transcript_biotype",           # Trans type
#            "description",                  # Gene description
#            "band",                         # Karyotype band
#            "refseq_mrna",
            "go_id",                        # Go Term accession (cellular domains)
#            "go_linkage_type",              # Go Term evidence code
            "name_1006"                    # Go Term name
#            "definition_1006",              # Go Term definition
#            "namespace_1003"                # Go domain                  
    ),
    values = rownames(sc@fdata),
    mart = mart
)

In [ ]:
dim(k)
head(k)

In [ ]:
# Our main marker genes of interest
# Note that "Brachyury" is not a gene name but is related to 'T' and 'T2' genes
mesoderm <- c("Eomes", "Mesp1" ,"T", "T2")
pluripotent <-c("Pou5f1", "Nanog")
neuroectoderm <-c("Sox1", "Sox2", "Pou3f1", "Zfp462", "Slc7a3")

# Find their ensembl ids and replace the names in sc@ndata
findID <- function(name){
    res <- (unique(k[k$external_gene_name == name,]$ensembl_gene_id))
    if (length(res) == 0){
        message("Missing: [",name, "] in Ensembl biomart")
    }
    return(res)
}

findID_array <- function(array){
    ids <- c()
    for (elem in array){
        e_id <- findID(elem)
        ids <- append(ids, e_id)
    }
    return(ids)
}

ids_exist_in_data <- function(obj, array){
    missing_genes = c()
    for (elem in array){
        res <- grep(elem, rownames(obj@fdata))
        if (length(res) == 0){
            missing_genes <- append(missing_genes, elem)
        }
    }
    if (length(missing_genes) > 0){
        message("Missing ", 
                length(missing_genes),"/", length(array),
                " [", paste(missing_genes, collapse=", "), 
                "] in filtered data")
    }
}

mesod_ids <- findID_array(mesoderm)
pluri_ids <- findID_array(pluripotent)
neuro_ids <- findID_array(neuroectoderm)

No warning messages means that we have indeed found the ensemble IDs for our genes of interest in the biomart. Let us see if this still persists after filtering.

In [ ]:
# Defaults
sc1 <- filterdata(
    sc, mintotal=3000, minexpr=5, maxexpr=500, 
    downsample = F, sfn = F, hkn = F,
    dsn = 1, rseed = 17000, CGenes = NULL, FGenes = NULL
)
message(dim(sc@fdata)[[1]], " genes and ", dim(sc@fdata)[[2]], " cells.")

ids_exist_in_data(sc1, mesod_ids)
ids_exist_in_data(sc1, pluri_ids)
ids_exist_in_data(sc1, neuro_ids)

In [ ]:
# Defaults, reduced minexpr 5→1
sc2 <- filterdata(
    sc, mintotal=3000, minexpr=1, maxexpr=500, 
    downsample = F, sfn = F, hkn = F,
    dsn = 1, rseed = 17000, CGenes = NULL, FGenes = NULL
)
message(dim(sc@fdata)[[1]], " genes and ", dim(sc@fdata)[[2]], " cells.")

ids_exist_in_data(sc2, mesod_ids)
ids_exist_in_data(sc2, pluri_ids)
ids_exist_in_data(sc2, neuro_ids)

In [ ]:
# Defaults, reduced minexpr 5→1, increase maxexpr 500→Inf
sc3 <- filterdata(
    sc, mintotal=4000, minexpr=1, maxexpr=Inf, 
    downsample = F, sfn = F, hkn = F,
    dsn = 1, rseed = 17000, CGenes = NULL, FGenes = NULL
)
message(dim(sc@fdata)[[1]], " genes and ", dim(sc@fdata)[[2]], " cells.")

ids_exist_in_data(sc3, mesod_ids)
ids_exist_in_data(sc3, pluri_ids)
ids_exist_in_data(sc3, neuro_ids)

In [ ]:
x <- CCcorrect(
    sc@fdata,
    vset=vset,
    CGenes=NULL, nComp=NULL,
    ccor=.4, pvalue=.05, quant=.01,
    mode="pca")

In [ ]:
x$n
# Inspect the loadings assosciated with each PC
#y <- x$pca$rotation[,x$n[1]]
#tail(y[order(y,decreating=T)],10)

`NULL` - The function runs properly as evidences by `x$pca` and `x$xcor`, but `x$n` yields nothing because there were likely no genes in our data known to have a significant effect towards Cell Proliferation or to the Cell Cycle effect.

We can still apply what little correction computed agains this affect onto our set. If we look below we can see what effect this would have (answer: none):

In [ ]:
message("Before correction")
head(sc@fdata)
message("After correction")
head(x$xcor)
message("No visible change...")

# K-mediods clustering

We will now perform the initial clustering with default settings.

In [ ]:
sc <- clustexp(
    sc, clustnr=30, bootnr=50, metric="pearson",
    do.gap=T, sat=TRUE, SE.method="Tibs2001SEmax",
    SE.factor=.25, B.gap=50,
    cln=0,rseed=17000, FUNcluster="kmedoids",FSelect=TRUE
)

Let us examine just how effective the clustering was and whether the clusters are stable.

In [ ]:
plotsaturation(sc, disp=T)

As we can see, there is a significant drop between k=1 and k=2, and then the addition of more clusters only leads to a linear decrease in the cluster-dispersion.

Circled is the cluster number that is inferred based upon the saturation of the average within-cluster dispersion.

Let us confirm this with this with the:
  1. Gap statistic, which once again shows a significant dispersion (outside of the error bars) between k=1 and k=2.
  2. Silhoutte plot, which defines the resolution of the clustering where each point is assigned a score based on the average similarity it has to each point in that cluster, and another score where it computes the same in comparison to all points in the closest neighbouring cluster. The difference in these scores takes on a value between -1 (bad) and 1 (good). Below we see only positive values in each cluster, representing good clustering.
  3. Jaccard similarty, which is the intersection over the union of two sets (or in this case, two neighbouring clusters). Stable clusters should have a similarity greater than 0.6. Below we see scores well above 0.8.

In [ ]:
plotgap(sc)
# silhouette of k-medoids clusters
plotsilhouette(sc)
# Jaccard's similarity of k-medoids clusters
plotjaccard(sc)

## Outlier cell identification

The distribution of counts in each gene is modelled as a negative binomial distribution, with the mean as the average gene expression across all cells in a cluster, and the dispersion is a second order log polynomial that describes the variability of a gene being expressed differently between different clusters (where the majority of biologically insignificant genes should have low variability).

Outlier cells are then identified based on their deviation from this model, where any cells with genes with a variability above this model are flagged.  



In [ ]:
sc <- findoutliers(sc, outminc=5, outlg=2, probthr=1e-3, 
                   thr=2**-(1:40), outdistquant=0.95)

In [ ]:
plotbackground(sc)

We can see here that the regression of the variance of the mean (red) fits the local regression (orange dashed), which likely indicates a lack of outlier types in our data.

In [ ]:
plotsensitivity(sc)
plotoutlierprobs(sc)

Here we can define our outlier probability cutoff as -log10(3), and plot this limit against the observed outlier probabilites observed for each cell in each cluster.

## Clustering: heatmap and tSNE

We can further examine the cell-to-cell distances with a clustering heatmap for both initial clustering and that derived from outlier detection

In [ ]:
x <- clustheatmap(sc, final=F, hmethod="single")
x <- clustheatmap(sc, final=T, hmethod="single")

sc <- comptsne(
    sc, rseed=15555, sammonmap=FALSE,
    initial_cmd=TRUE,fast=TRUE,perplexity=30
)

plottsne(sc, final = F)
plottsne(sc, final = T)

Outlier cells are identified in each of the two main clusters but we can see that there does not seem to be any significant outlier clusters due to how spread out they appear.

## Applying Random forest correction to cell outliers

Cells are reclassified into clusters, using RF method to test for robustness of the original clustering. We can see with a heatmap that the final clustering has reduced some of original 


In [ ]:
sc <- rfcorrect(sc, rfseed=12345, final=T, nbfactor=5)

In [ ]:
x <- clustheatmap(sc, final=T, hmethod="single")
plottsne(sc, final = T)

## Repeating the random  forest correction to see if there is improvement in clustering.

We will try again, this time with a different seed and a significantly increased number of trees (500) to see if there is better outlier clustering.


In [ ]:
sc <- rfcorrect(sc, rfseed=12345, final=T, nbfactor=500)

In [ ]:
x <- clustheatmap(sc, final=T, hmethod="single")
plottsne(sc, final = T)

Once again cells assigned to a given outlier cluster seem to be quite displaced from one another, although there does seem to be a consistency in the outlier cells being on the fringes of the two main clusters.

### Marker Gene analysis

Let us examine our marker genes of interest to see if there is any pattern of expression that is consistent with the observed clustering.

In [ ]:

#grep("Brachyury", (unique(k$external_gene_name)))
#tail(k)
#grep("ENSMUSG00000032446", rownames(sc@fdata))
#grep("ENSMUSG00000110344", rownames(sc@expdata))
head(k)

In [ ]:
# Our main marker genes of interest
mesoderm <- c("Eomes", "Brachyury", "Mesp1")
pluripotent <-c("Pou5f1", "nanog")
neuroectoderm <-c("Sox1", "Sox2", "Pou3f1", "zfp462", "slc7a3")

# Find their ensembl ids and replace the names in sc@ndata
findID <- function(name){
    res <- (unique(k[k$external_gene_name == name,]$ensembl_gene_id))
    if (length(res) == 0){
        message("Missing: [",name, "]: most likely was filtered out")
    }
    return(res)
}

findID_array <- function(array){
    ids <- c()
    for (elem in array){
        e_id <- findID(elem)
        ids <- append(ids, e_id)
    }
    return(ids)
}


findID_array(mesoderm)
findID_array(pluripotent)
findID_array(neuroectoderm)


It appears that our genes of interest were most likely filtered out during the normalisation/filtering process, so either these genes are not biologically significant, OR our filtering was too stringent.

We will repeat the analysis using relaxed filtering criterion.

In [ ]:
#plotexptsne(sc,mesoderm,n="Mesoderm genes",logsc=TRUE)
#plotexptsne(sc,pluripotent,n="Pluripotency genes",logsc=TRUE)
#plotexptsne(sc,neuroectoderm,n="Neuro-ectoderm genes",logsc=TRUE)